# Miary
---
obliczenie miar zagregowanych (X) dla całej sieci

korzysta z plików .csv obliczonych w pliku `obliczenia.ipynb`

In [12]:
import pandas as pd
import numpy as np

#### parametrty do scenariuszy

In [13]:
Bs= [60, 75, 90, 105, 120] 
t_ks =[30,40,50,60,70,80,90]
modes = ["PT","Car"]
aggs = ['mean','min','max','std','median', 'p_25', 'p_75']

In [14]:
def readone(ch,B,t_k,mode, agg, folder = 'data//res//'):
    """ otworz odpowiedni plik csv i załaduj do pandas"""
    name = "{}_{}_{}_{}".format(ch,mode,str(B),str(t_k))
    path = folder+'A_i_'+name+".csv"  
    df = pd.read_csv(path, index_col=0)
    return df['A_i_'+name+agg]

def X(ch,B,t_k):
    """ otworz dla pliki csv i oblicz miatę X dla odpowiednich parametrów"""
    a = readone(ch,B,t_k,modes[0],aggs[0])
    b = readone(ch,B,t_k,modes[1],aggs[0])
    i = a.index.union(b.index)
    a = a.reindex(i).fillna(0)
    b = b.reindex(i).fillna(0)
    df =  pd.DataFrame((a - b)/(a  + b ))
    df.columns=['mean']
    for agg in aggs[1:]:
        a = readone(ch,B,t_k,modes[0],agg)
        b = readone(ch,B,t_k,modes[1],agg)
        i = a.index.union(b.index)
        a = a.reindex(i).fillna(0)
        b = b.reindex(i).fillna(0)
        df[agg] = (a - b)/(a  + b )
    return df
    

#### pętla dla wszysctkich scenariuszy 
* załaduj pliki
* oblicz miarę
* zapisz plik

In [15]:
for ch in ["C","H"]:
    for B in Bs:
        for t_k in t_ks:
            if B>t_k:
                folder = 'data//res//'
                name = "{}_{}_{}".format(ch,str(B),str(t_k))
                path = folder+'X_'+name+".csv" 
                X(ch,B,t_k).to_csv(path, sep = ";")
                

### 4.	Regional modal accessibility disparity, unweighted

In [17]:
res = pd.DataFrame(columns=['ch','B','t_k', 'mode', 'agg', 'PT','Car','X'])
for ch in ["C","H"]:
    for B in Bs:
            for t_k in t_ks:
                if B>t_k: 
                    for agg in aggs:
                        PT = readone(ch,B,t_k, 'PT', agg).mean()
                        Car =readone(ch,B,t_k, 'Car', agg).mean()
                        X = (PT - Car)/(PT + Car)
                        res.loc[res.shape[0]+1]=[ch,B,t_k, mode, agg,PT, Car,X]
res.to_csv('data//res//zbiorcze.csv')

In [18]:
res

,ch,B,t_k,mode,agg,PT,Car,X
1,C,60,30,Car,mean,34.479193,33.664040,0.011962
2,C,60,30,Car,min,30.287060,30.032646,0.004218
3,C,60,30,Car,max,48.989187,44.140344,0.052066
4,C,60,30,Car,std,4.104868,2.883291,0.174807
5,C,60,30,Car,median,33.268431,33.016526,0.003800
6,C,60,30,Car,p_25,31.557064,31.360160,0.003130
7,C,60,30,Car,p_75,36.181494,35.355183,0.011551
8,C,60,40,Car,mean,43.752965,41.594522,0.025290
9,C,60,40,Car,min,41.110415,40.176137,0.011494
10,C,60,40,Car,max,49.718397,44.689678,0.053266


----